In [2]:
!pip install Keras #Keras-2.8.0

     |████████████████████████████████| 1.4 MB 125 kB/s eta 0:00:01


In [4]:
!pip install tensorflow


     |████████████████████████████████| 217.5 MB 39 kB/s  eta 0:00:01
     |████████████████████████████████| 42 kB 1.3 MB/s eta 0:00:01
     |████████████████████████████████| 3.1 MB 2.1 MB/s eta 0:00:01
     |████████████████████████████████| 462 kB 2.3 MB/s eta 0:00:01
     |████████████████████████████████| 1.6 MB 2.0 MB/s eta 0:00:01
     |████████████████████████████████| 57 kB 1.7 MB/s eta 0:00:01
     |████████████████████████████████| 13.0 MB 2.7 MB/s eta 0:00:01
     |████████████████████████████████| 65 kB 1.5 MB/s eta 0:00:01
     |████████████████████████████████| 4.3 MB 2.5 MB/s eta 0:00:01     |███████▌                        | 993 kB 2.2 MB/s eta 0:00:02     |███████████████████▍            | 2.6 MB 2.5 MB/s eta 0:00:01
     |████████████████████████████████| 126 kB 2.2 MB/s eta 0:00:01
     |████████████████████████████████| 5.8 MB 2.1 MB/s eta 0:00:01
     |████████████████████████████████| 289 kB 2.2 MB/s eta 0:00:01
     |████████████████████████████████| 156 kB 3.9

In [5]:
import json
import pandas as pd
import numpy as np
from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
import scipy.stats as st
from sklearn.model_selection import train_test_split
from datetime import datetime
from sklearn import preprocessing
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, confusion_matrix, adjusted_rand_score
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import neighbors
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.linear_model import LogisticRegression

In [6]:
# Data loading option 2: from local file
import pickle
import os

def load(file_name):
    with open(file_name, 'rb') as fp:
        obj = pickle.load(fp)
    return obj

path = os.getcwd() #'/Users/fuad/Documents/GitHub/models'

In [7]:
# set project folder location where the models and data folder exist
projectlocation = '/Users/fuad/Documents/GitHub'

In [8]:
#set data path and load data
datapath = projectlocation + "/data/flare_prediction_mvts_data.pck"
labelpath = projectlocation + "/data/flare_prediction_labels.pck"

trainData = load(datapath)
trainLebel = load(labelpath)

print("trainData.shape: ",trainData.shape)
print("trainLebel.shape: ",trainLebel.shape)

trainData.shape:  (1540, 33, 60)
trainLebel.shape:  (1540,)


In [9]:
#four-class problem {X, M, B/C, Q}
print("np.unique(trainLebel): ",np.unique(trainLebel))

np.unique(trainLebel):  [0 1 2 3]


In [23]:
#Binary classification
# Data label conversion to BINARY class
def get_binary_labels_from(labels_str):
    tdf = pd.DataFrame(labels_str, columns = ['labels'])
    data_classes= [0, 1, 2, 3]
    d = dict(zip(data_classes, [0, 0, 1, 1])) 
    arr = tdf['labels'].map(d, na_action='ignore')
    return arr.to_numpy()

#un-comment next line for Binary classification experiment
trainLebel = get_binary_labels_from(trainLebel)

In [11]:
temptrainData=np.empty([1540,60, 33])
n=len(trainData)
for l in range(0, n):
  temp=trainData[l]
  #print(temp)
  #temp=np.transpose(temp)
  temp=temp.T
  #print(temp.shape)
  #print(temp)
  temptrainData[l,:,:]=temp
  n=n+1
  #np.append(temptrainData, temp)
  #print(temptrainData)

#print(temptrainData.shape)
#print(trainData.shape) 
trainData=temptrainData
print("trainData.shape: ",trainData.shape)
#print(trainData[0])

temp=trainData[0]
#print(temp)
df = pd.DataFrame(temp)
#df=pd.DataFrame.from_dict(trainData)
trainData222=trainData

df

trainData.shape:  (1540, 60, 33)


,0,1,2,3,4,5,6,7,8,9,...,23,24,25,26,27,28,29,30,31,32
0,6748.177901,8.294421e+10,1.591225e+24,1.095967e+14,1510.190735,5.537032e+13,6.524454e+22,-1.945966e+25,13461.392092,-0.176659,...,5.414326,5.392323,5.336158,5.205328,0.0,0.0,0.000000,0.000000,1424.0,0.000004
1,6713.245495,8.301113e+10,1.591180e+24,1.082126e+14,1552.899870,5.163912e+13,6.552311e+22,-1.967367e+25,13459.043822,-0.178458,...,5.405846,5.403088,5.335880,5.211822,0.0,0.0,0.000000,0.000000,1400.0,0.000007
2,6711.746592,8.324237e+10,1.585508e+24,1.074454e+14,1569.593570,5.208053e+13,6.582194e+22,-2.022283e+25,13391.359484,-0.182930,...,5.405772,5.398195,5.345981,5.216945,0.0,0.0,0.000000,0.000039,1405.0,0.000003
3,6903.974288,8.343257e+10,1.576556e+24,1.084795e+14,1608.218901,5.166028e+13,6.609184e+22,-2.072753e+25,13248.436229,-0.187068,...,5.396873,5.398732,5.327560,5.203755,0.0,0.0,0.000000,0.000000,1429.0,0.000003
4,6691.365852,8.370511e+10,1.600757e+24,1.086643e+14,1479.272564,5.030951e+13,6.614713e+22,-1.999304e+25,13367.513454,-0.179852,...,5.426061,5.389834,5.351214,5.217820,0.0,0.0,0.000000,0.000000,1430.0,0.000003
5,6834.958217,8.406158e+10,1.591036e+24,1.096684e+14,1644.488148,5.663543e+13,6.630166e+22,-2.075205e+25,13298.131406,-0.185888,...,5.421926,5.407266,5.350388,5.219855,0.0,0.0,0.000000,0.000000,1517.0,0.000004
6,6830.348029,8.431275e+10,1.589851e+24,1.083214e+14,1587.280389,5.648788e+13,6.649235e+22,-2.105215e+25,13390.341243,-0.188014,...,5.419279,5.413495,5.353073,5.232006,0.0,0.0,0.000039,0.000000,1477.0,0.000004
7,6894.216660,8.429531e+10,1.577024e+24,1.090471e+14,1530.971594,5.153441e+13,6.685179e+22,-2.175302e+25,13251.479423,-0.194314,...,5.413199,5.418618,5.341884,5.221831,0.0,0.0,0.000000,0.000000,1599.0,0.000015
8,6906.079329,8.449927e+10,1.586829e+24,1.088327e+14,1580.642155,5.204076e+13,6.646028e+22,-2.155312e+25,13485.254521,-0.192063,...,5.419249,5.426273,5.354147,5.243087,0.0,0.0,0.000000,0.000000,1585.0,0.000008
9,6899.094797,8.451908e+10,1.601676e+24,1.091217e+14,1562.139216,5.105149e+13,6.593766e+22,-2.084657e+25,13729.350076,-0.185724,...,5.427837,5.393532,5.321879,5.221579,0.0,0.0,0.000000,0.000000,1726.0,0.000005


In [12]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
print(trainData.shape)

print(type(trainData))
npArrays=[]
for l in range(0, len(trainData)):
  trainData_std = sc.fit_transform(trainData[l])
  npArrays.append(trainData_std)

print(type(npArrays))
arr = np.asarray(npArrays)
print(type(arr))
trainData_scaled=arr
df = pd.DataFrame(trainData_scaled[0])

(1540, 60, 33)
<class 'numpy.ndarray'>
<class 'list'>
<class 'numpy.ndarray'>


In [13]:
num_mvts=trainData_scaled.shape[0]
num_ts=trainData_scaled.shape[1]
num_params=trainData_scaled.shape[2]

#num_mvts=2
#num_mvts=len(trainData_scaled)
x = np.zeros( (num_mvts,num_params*8) )
print(x.shape)

(1540, 264)


In [14]:
for i in range(0, num_mvts):
  #print("mvts: ",i)
  temp=trainData_scaled[i]
  

  temp_X = np.zeros( (num_params,8) )

  for j in range(0, num_params):
    #print("param: ",j)
    ts = trainData_scaled[i,:, j]
    #print("ts length: ",ts.shape)

    v1 = np.mean(ts)
    v2 = np.std(ts)
    v3 = st.skew(ts)
    v4 = st.kurtosis(ts)

    ts_p = np.zeros((len(ts) - 1))
    for k in range(len(ts_p)):
       ts_p[k] = ts[k + 1] - ts[k]
       
    v5 = np.mean(ts_p)
    v6 = np.std(ts_p)
    v7 = st.skew(ts_p)
    v8 = st.kurtosis(ts_p)
    vect_ts = [v1, v2, v3, v4, v5, v6, v7, v8]
    #X[i, j * 8:j * 8 + 8] = vect_ts
    #print(col, " vect_ts: ",vect_ts)
    temp_X[j,:] = vect_ts
    #print("x[",col,"]=",X[col])
  #print("temp_X: ",temp_X.shape)
  #print(X)

  row = temp_X.reshape((num_params*8))
  x[i,:]=row

  #x2=temp_X[0]
  #for row in range(1, colNumber):
    #x2=np.concatenate((x2,temp_X[1]), axis=0)
  #print("x2.shape: ",x2.shape)
  #x[table] = x2


In [15]:
trainData_8n=x
#print("x.shape: ",x.shape) 
print("trainData_8n.shape: ",trainData_8n.shape)

temp=trainData_8n[0]
#print(temp)
df = pd.DataFrame(temp)
#df=pd.DataFrame.from_dict(trainData_8n)

df


trainData_8n.shape:  (1540, 264)


,0
0,1.202742e-14
1,1.000000e+00
2,4.577300e-01
3,-8.984097e-01
4,3.050571e-02
...,...
259,7.335960e+00
260,1.415458e-02
261,1.093650e+00
262,5.326185e-01


In [16]:
import random

from sklearn.metrics import roc_auc_score

def doBaseLineMethodsBasedCalcultions(inputData, X_train, X_test, y_train, y_test):
  num_masterIteration=5

  #inputData=trainData_8n

  #print(type(inputData))
  #print(type(inputData[0]))
  #print(inputData.shape)

  classification_report_dict=[]
  #Accuracy=[][]
  Accuracy=[]
  roc_auc=[]
  for masterIteration in range(num_masterIteration):
      print("\nmasterIteration: ",masterIteration)
      #print(bcolors.WARNING + "\nmasterIteration :" + bcolors.WARNING,masterIteration)
      svm_model = SVC()
      rf_model = RandomForestClassifier()
      #forest = RandomForestClassifier(criterion='entropy',n_estimators=10, random_state=1, n_jobs=2)
      nb_model = GaussianNB()
      dt = DecisionTreeClassifier(criterion='entropy', random_state=0)
      knn = neighbors.KNeighborsClassifier(n_neighbors=2)

      # all parameters not specified are set to their defaults
      #logisticRegr = LogisticRegression()
      #logisticRegr = LogisticRegression(max_iter=1000)
      #https://stackoverflow.com/questions/52670012/convergencewarning-liblinear-failed-to-converge-increase-the-number-of-iterati
      logisticRegr = LogisticRegression(solver='lbfgs',class_weight='balanced', max_iter=4000)

      #tree = DecisionTreeClassifier( random_state=23)
      #adaboost = AdaBoostClassifier(base_estimator=tree, n_estimators=5, learning_rate=0.1, random_state=23)
      #bagging = BaggingClassifier(base_estimator=tree, n_estimators=5, max_samples=50, bootstrap=True)

      models = [svm_model, rf_model, nb_model,dt,knn]
      #models = [logisticRegr]
      for i in range(len(models)):
          mod = models[i]
          model_name = type(mod).__name__
          #print( model_name)
          mod.fit(X_train, y_train)

          #print(f"Train score: {mod.score(X_train, y_train)}")
          #print(f"Test score: {mod.score(X_test, y_test)}")

          y_pred = mod.predict(X_test)
          aaafsfsfsf=accuracy_score(y_test, y_pred)
          #print(model_name, ' Accuracy: %.4f' %aaafsfsfsf )
          Accuracy.append(aaafsfsfsf)

          sfsgsg=roc_auc_score(y_test, y_pred)
          roc_auc.append(sfsgsg)
          #print(model_name, ' roc_auc: %.4f' %sfsgsg )
          #print('Adjusted Accuracy : %.3f' % adjusted_rand_score(labels_true=y_test, labels_pred=y_pred))
          #print("classification_report:\n ", classification_report(y_test, y_pred))
        

          #TN, FP, FN, TP = 
          confusion_matrix=metrics.confusion_matrix(y_test, y_pred,labels=np.unique(trainLebel))
          #print(model_name,'Confusion Matrix : \n', confusion_matrix)
          #print(type(y_test))
          #print(type(y_pred))
          sfsfsf2=metrics.classification_report(y_test, y_pred, digits=3)
  
          print(model_name,'classification_report : \n',sfsfsf2)

          sfsfsf=metrics.classification_report(y_test, y_pred, digits=3,output_dict=True)
          classification_report_dict.append(sfsfsf)
          
          #print(model_name,'classification_report_dict : \n',classification_report_dict)
      #print('roc_auc : \n',roc_auc)
  doClassSpecificCalulcation(roc_auc,Accuracy,trainLebel,classification_report_dict)


In [17]:
def doClassSpecificCalulcation(roc_auc,Accuracy,trainLebel,classification_report_dict):
  print("\n\n\n *************** Final Report ***************")
  print('\np.mean(roc_auc) :',np.mean(roc_auc))
  print('\np.std(roc_auc) :',np.std(roc_auc))
  print('\np.mean(Accuracy) :',np.mean(Accuracy))
  print('\np.std(Accuracy) :',np.std(Accuracy))
  print('\n33333333 p.mean np.std(Accuracy) :     ',np.round(np.mean(Accuracy),2),"+-",np.std(Accuracy) )
  for j in range( len(np.unique(trainLebel)) ):
    print('\n\n\n\nclass :',j) 
    precision=[]
    recall=[]
    f1_score=[]
    for i in range(len(classification_report_dict)):
      report=classification_report_dict[i]
      #print('classification_report : \n',report) 
      temp=report[str(j)]['precision'] 
      precision.append(temp)

      temp=report[str(j)]['recall'] 
      recall.append(temp)

      temp=report[str(j)]['f1-score'] 
      f1_score.append(temp)

    print('\np.mean(precision) \t p.mean(recall) \t p.mean(f1_score) :') 


    print(np.mean(precision)) 
    print(np.mean(recall)) 
    print(np.mean(f1_score))

    print('\np.mean p.std(precision) \tp.mean  p.std(recall) \tp.mean  p.std(f1_score) :')

    print(np.round(np.mean(precision),2),"+-",np.std(precision) )
    print(np.round(np.mean(recall),2),"+-",np.std(recall) )
    print(np.round(np.mean(f1_score),2),"+-",np.std(f1_score) )


In [18]:
print(trainData_scaled.shape)
n=len(trainData_scaled)
lastRows=[]
for l in range(0, n):
  temp = trainData_scaled[l]
  #print(temp.shape)
  lastROw=temp[trainData_scaled.shape[1]-1]
  #print(lastROw.shape)
  #print(lastROw)
  #print(type(lastROw))
  lastRows.append(lastROw)

#print(lastRows)
df_lastRows = pd.DataFrame(lastRows)

lastRows = np.array(lastRows)

print(lastRows.shape)

print(trainData_scaled.shape)
n=len(trainData_scaled)
flatten_tables=[]
for l in range(0, n):
  temp = trainData_scaled[l]
  flatten_temp=temp.ravel()
  #print(flatten_temp.shape)
  #print(flatten_temp)
  #print(type(flatten_temp))
  flatten_tables.append(flatten_temp)

df_flatten_tables = pd.DataFrame(flatten_tables)

flatten_tables = np.array(flatten_tables)

print(flatten_tables.shape)


(1540, 60, 33)
(1540, 33)
(1540, 60, 33)
(1540, 1980)


In [19]:
def startCalculations(inputData):
  #test_sizes=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
  test_sizes=[0.3]
  for temp4 in range(len(test_sizes)):
    test_size=test_sizes[temp4]
    print("\n\n\n *************** test_size: ",test_size)
    random_state=0#random.randint(42, 100)
    print("random_state: ",random_state)
    X_train, X_test, y_train, y_test = train_test_split(inputData, trainLebel, 
                                                            test_size=test_size, 
                                                            random_state=random_state)
    print("X_train.shape X_test.shape y_train.shape y_test.shape ",
              X_train.shape, X_test.shape ,y_train.shape, y_test.shape)
    doBaseLineMethodsBasedCalcultions(inputData, X_train, X_test, y_train, y_test)


In [24]:
startCalculations(lastRows)##(1540, 33)




 *************** test_size:  0.3
random_state:  0
X_train.shape X_test.shape y_train.shape y_test.shape  (1078, 33) (462, 33) (1078,) (462,)

masterIteration:  0
SVC classification_report : 
               precision    recall  f1-score   support

           0      0.689     0.732     0.710       239
           1      0.692     0.646     0.668       223

    accuracy                          0.690       462
   macro avg      0.691     0.689     0.689       462
weighted avg      0.691     0.690     0.690       462

RandomForestClassifier classification_report : 
               precision    recall  f1-score   support

           0      0.687     0.762     0.722       239
           1      0.711     0.628     0.667       223

    accuracy                          0.697       462
   macro avg      0.699     0.695     0.694       462
weighted avg      0.698     0.697     0.695       462

GaussianNB classification_report : 
               precision    recall  f1-score   support

          

In [27]:
startCalculations(trainData_8n)




 *************** test_size:  0.3
random_state:  0
X_train.shape X_test.shape y_train.shape y_test.shape  (1078, 264) (462, 264) (1078,) (462,)

masterIteration:  0
SVC classification_report : 
               precision    recall  f1-score   support

           0      0.686     0.895     0.777       239
           1      0.833     0.561     0.670       223

    accuracy                          0.734       462
   macro avg      0.760     0.728     0.724       462
weighted avg      0.757     0.734     0.725       462

RandomForestClassifier classification_report : 
               precision    recall  f1-score   support

           0      0.804     0.941     0.867       239
           1      0.923     0.753     0.830       223

    accuracy                          0.851       462
   macro avg      0.863     0.847     0.848       462
weighted avg      0.861     0.851     0.849       462

GaussianNB classification_report : 
               precision    recall  f1-score   support

        

In [25]:
startCalculations(df_flatten_tables)




 *************** test_size:  0.3
random_state:  0
X_train.shape X_test.shape y_train.shape y_test.shape  (1078, 1980) (462, 1980) (1078,) (462,)

masterIteration:  0
SVC classification_report : 
               precision    recall  f1-score   support

           0      0.702     0.741     0.721       239
           1      0.705     0.664     0.684       223

    accuracy                          0.703       462
   macro avg      0.704     0.702     0.702       462
weighted avg      0.704     0.703     0.703       462

RandomForestClassifier classification_report : 
               precision    recall  f1-score   support

           0      0.677     0.808     0.737       239
           1      0.740     0.587     0.655       223

    accuracy                          0.701       462
   macro avg      0.709     0.697     0.696       462
weighted avg      0.708     0.701     0.697       462

GaussianNB classification_report : 
               precision    recall  f1-score   support

      